<a href="https://colab.research.google.com/github/QuantumSufi/UFCF-ZERO/blob/main/mini_lattice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mini_lattice.ipynb  – SU(2) 16^3x32 örnek demo
import torch, math, time
L, Nt, beta = 16, 32, 2.2          # örgü boyutu ve beta
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Basit link değişkeni: 2x2 birim matrisler + fraktal faz
def random_u2():
    theta = torch.rand(1, device=device) * 2*math.pi
    return torch.stack([torch.cos(theta), torch.sin(theta)], dim=-1)

U = torch.stack([[random_u2() for _ in range(4)] for _ in range(L**3*Nt)])

def staple_sum(U, idx):       # çok sadeleştirilmiş
    return sum(U[(idx+k) % U.shape[0]] for k in (1,2,3))

def sweep(U):
    accept = 0
    for idx in range(U.shape[0]):
        old = U[idx].clone()
        proposal = random_u2()
        dS = -beta * ( (proposal - old) * staple_sum(U, idx) ).sum()
        if dS < 0 or torch.rand(1, device=device) < torch.exp(-dS):
            U[idx] = proposal
            accept += 1
    return accept / U.shape[0]

# Isıl dengeleme
for it in range(50):
    acc = sweep(U)
    if it % 10 == 0:
        print(f'Iter {it}: acc={acc:.2f}')

# Plaquette ölçümü (çok kaba)
plaq = U.mean().item()
print("⟨P⟩ =", plaq)

# Kaydet
import numpy as np, os
os.makedirs('out', exist_ok=True)
np.savez('out/mini_lattice_results.npz', plaquette=plaq, beta=beta)
